In [100]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [101]:

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [102]:
cap = cv2.VideoCapture(0)
ALERT_DURATION = 15  
closed_eyes_counter = 0
threshold = 120 
alert_threshold = 15

In [103]:
model = load_model('drowsiness_model.h5')  

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))


    focus_status = "Not Focused"
    eye_open = False

    for (fx, fy, fw, fh) in faces:

        face_color = (0, 255, 0) if eye_open else (0, 0, 255)
        cv2.rectangle(frame, (fx, fy), (fx + fw, fy + fh), face_color, 2)


        roi_gray = gray[fy:fy + fh, fx:fx + fw]
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=3)

        for (ex, ey, ew, eh) in eyes:
            if ew < 25 or eh < 25:
                continue

            eye_roi = roi_gray[ey:ey + eh, ex:ex + ew]
            eye_roi = cv2.resize(eye_roi, (48, 48))
            eye_roi = eye_roi.astype("float32") / 255.0
            eye_roi = np.expand_dims(eye_roi, axis=-1)
            eye_roi = np.expand_dims(eye_roi, axis=0)

            prediction = model.predict(eye_roi, verbose=0)[0]
            print(prediction)
            open_prob = prediction[1]

            if open_prob > 0.6:
                eye_open = True


    if eye_open:
        focus_status = "Focused"
        closed_eyes_counter = 0
    else:
        closed_eyes_counter += 1


    cv2.putText(frame, f"Focus: {focus_status}", (20, 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 255, 0) if focus_status == "Focused" else (0, 0, 255), 2)


    if closed_eyes_counter >= ALERT_DURATION:
        cv2.putText(frame, "DROWSINESS ALERT!", (20, 100),  
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


    cv2.imshow("Drowsiness Detector (Face + Eyes)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[5.1781673e-05 9.9994826e-01]
[1.3381164e-05 9.9998665e-01]
[9.6820986e-06 9.9999034e-01]
[3.660193e-06 9.999963e-01]
[1.1691087e-04 9.9988306e-01]
[3.5854115e-05 9.9996412e-01]
[2.1906137e-04 9.9978095e-01]
[2.3283004e-05 9.9997675e-01]
[1.0123531e-06 9.9999905e-01]
[3.712824e-05 9.999629e-01]
[1.5671310e-06 9.9999845e-01]
[4.0620671e-06 9.9999595e-01]
[3.7398699e-05 9.9996257e-01]
[3.5526526e-05 9.9996448e-01]
[0.001026   0.99897397]
[1.403299e-04 9.998597e-01]
[8.137005e-05 9.999186e-01]
[1.3747587e-05 9.9998629e-01]
[8.224934e-06 9.999918e-01]
[3.285578e-05 9.999671e-01]
[1.8218834e-05 9.9998176e-01]
[1.13664064e-04 9.99886274e-01]
[5.8178551e-04 9.9941826e-01]
[0.6068319 0.3931681]
[0.01803316 0.98196685]
[9.994991e-01 5.009582e-04]
[9.9987829e-01 1.2169149e-04]
[0.02517482 0.9748252 ]
[3.005238e-06 9.999970e-01]
[0.37295803 0.62704194]
[0.10446304 0.8955369 ]
[0.00298585 0.99701416]
[6.2670624e-05 9.9993730e-01]
[0.55615747 0.44384253]
[0.48812282 0.51187724]
[0.00658594 0.993414